In [1]:
from thinc.api import IvyWrapper, Adam

/Users/yog/miniconda3/envs/thinc/lib/python3.10/site-packages/ivy/utils/exceptions.py:415: UserWarning: The current backend: 'jax' does not support inplace updates natively. Ivy would quietly create new arrays when using inplace updates with this backend, leading to memory overhead (same applies for views). If you want to control your memory management, consider doing ivy.set_inplace_mode('strict') which should raise an error whenever an inplace update is attempted with this backend.
  warnings.warn(


In [2]:
import ivy

In [3]:
import ml_datasets
from tqdm import tqdm

In [4]:
(train_X, train_Y), (dev_X, dev_Y) = ml_datasets.mnist()
print(f"Training size={len(train_X)}, dev size={len(dev_X)}")

Training size=54000, dev size=10000


In [5]:
def train_model(data, model, optimizer, n_iter, batch_size):
    (train_X, train_Y), (dev_X, dev_Y) = data
    indices = model.ops.xp.arange(train_X.shape[0], dtype="i")
    for i in range(n_iter):
        batches = model.ops.multibatch(batch_size, train_X, train_Y, shuffle=True)
        for X, Y in tqdm(batches, leave=False):
            Yh, backprop = model.begin_update(X)
            backprop(Yh - Y)
            model.finish_update(optimizer)
        # Evaluate and print progress
        correct = 0
        total = 0
        for X, Y in model.ops.multibatch(batch_size, dev_X, dev_Y):
            Yh = model.predict(X)
            correct += (Yh.argmax(axis=1) == Y.argmax(axis=1)).sum()
            total += Yh.shape[0]
        score = correct / total
        print(f" {i} {float(score):.3f}")

In [6]:
width = 32
nO = 10
nI = 784
dropout = 0.2

In [7]:
class Regressor(ivy.Module):
    def __init__(self, width, nO, nI, dropout):
        self.nO = nO
        self.nI = nI
        self.dropout = dropout
        super().__init__()

    def _build(self, *args, **kwargs):
        self.dropout1 = ivy.Dropout(self.dropout)
        self.dropout2 = ivy.Dropout(self.dropout)
        self.fc1 = ivy.Linear(self.nI, width)
        self.fc2 = ivy.Linear(width, self.nO)

    def _forward(self, x):
        x = ivy.relu(x)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = ivy.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = ivy.log_softmax(x, axis=1)
        return output

In [8]:
wrapped_model = IvyWrapper(Regressor(width, nO, nI, dropout))

/Users/yog/miniconda3/envs/thinc/lib/python3.10/site-packages/ivy/utils/exceptions.py:415: UserWarning: The current backend: 'jax' does not support inplace updates natively. Ivy would quietly create new arrays when using inplace updates with this backend, leading to memory overhead (same applies for views). If you want to control your memory management, consider doing ivy.set_inplace_mode('strict') which should raise an error whenever an inplace update is attempted with this backend.
  warnings.warn(


In [9]:
wrapped_model.shims[0]._model.v

/Users/yog/miniconda3/envs/thinc/lib/python3.10/site-packages/ivy/utils/exceptions.py:415: UserWarning: The current backend: 'jax' does not support inplace updates natively. Ivy would quietly create new arrays when using inplace updates with this backend, leading to memory overhead (same applies for views). If you want to control your memory management, consider doing ivy.set_inplace_mode('strict') which should raise an error whenever an inplace update is attempted with this backend.
  warnings.warn(


{
    fc1: {
        b: (<class ivy.data_classes.array.array.Array> shape=[32]),
        w: (<class ivy.data_classes.array.array.Array> shape=[32, 784])
    },
    fc2: {
        b: ivy.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
        w: (<class ivy.data_classes.array.array.Array> shape=[10, 32])
    }
}

In [10]:
data = ml_datasets.mnist()
optimizer = Adam(0.001)
wrapped_model.initialize(X=train_X[:5], Y=train_Y[:5])
train_model(data, wrapped_model, optimizer, n_iter=10, batch_size=128)

 0 0.124


 1 0.122


 2 0.127


 3 0.122


 4 0.125


 5 0.119


 6 0.125


 7 0.127


 8 0.126


 9 0.122
